# Розширення генерації на основі пошуку (RAG) та векторні бази даних

In [ ]:
%pip install getenv openai faiss-cpu pandas numpy

In [42]:
import os
import pandas as pd
import numpy as np
import faiss

from dotenv import load_dotenv

load_dotenv()

True

## Створення нашої бази знань

Налаштування FAISS для векторного пошуку


In [ ]:
# Шляхи до файлів (даних для RAG)
data_paths = [
    "data/frameworks.md", 
    "data/own_framework.md", 
    "data/perceptron.md"
] 

# Ініціалізація порожнього DataFrame
df = pd.DataFrame(columns=['path', 'text'])

# Сучасний спосіб додавання рядків до DataFrame
for path in data_paths:
    try:
        with open(path, 'r', encoding='utf-8') as file:
            file_content = file.read()
        
        # Використовуємо concat замість застарілого append
        new_row = pd.DataFrame({'path': [path], 'text': [file_content]})
        df = pd.concat([df, new_row], ignore_index=True)
    except FileNotFoundError:
        print(f"Файл не знайдено: {path}")

df.head()

In [ ]:
def split_text(text, max_length, min_length):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(' '.join(current_chunk)) < max_length and len(' '.join(current_chunk)) > min_length:
            chunks.append(' '.join(current_chunk))
            current_chunk = []

    # Якщо останній фрагмент не досягнув мінімальної довжини, все одно додати його
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Припускаючи, що analyzed_df - це pandas DataFrame, а 'output_content' - це стовпець у цьому DataFrame
splitted_df = df.copy()
splitted_df['chunks'] = splitted_df['text'].apply(lambda x: split_text(x, 400, 300))

splitted_df

In [ ]:
# Припускаючи, що 'chunks' - це стовпець списків у DataFrame splitted_df, ми розділимо фрагменти на різні рядки
flattened_df = splitted_df.explode('chunks')

flattened_df.head()

## Перетворення тексту на ембедінги

In [70]:
from azure.ai.inference import EmbeddingsClient
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
token = os.environ["GITHUB_TOKEN"]

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

def create_embeddings(text):
    """
    Створює ембедінги для тексту.
    
    Args:
        text: Текст або список текстів для ембедінгу
        model: Модель для ембедінгу (за замовчуванням mistral-embed)
        
    Returns:
        Вектор ембедінгу
    """
    # Обробка pandas Series
    if isinstance(text, pd.Series):
        # Беремо перший елемент з Series
        text = text.iloc[0]
    
    # Перетворюємо в список рядків для API
    if not isinstance(text, list):
        text = [str(text)]
    else:
        text = [str(item) for item in text]
    
    embeddings_response = embed_client.embed(
        input=text,
        model=embed_model_name
    )
    
    # Повернення ембедінгу для першого елемента
    return embeddings_response.data[0].embedding

# Приклад використання:
embeddings = create_embeddings(flattened_df['chunks'][0])

[0.00630188,
 0.010032654,
 -0.0020694733,
 -0.015914917,
 -0.02796936,
 0.007827759,
 -0.005268097,
 -0.036468506,
 -0.005508423,
 0.0060577393,
 0.023132324,
 0.013717651,
 0.0005393028,
 0.010406494,
 0.005924225,
 0.009422302,
 0.03390503,
 -0.021362305,
 0.010467529,
 -0.011383057,
 -0.0012874603,
 0.018737793,
 0.042938232,
 0.0024642944,
 -0.036132812,
 0.043304443,
 0.016082764,
 -0.036895752,
 0.018218994,
 -0.029144287,
 -0.016052246,
 -0.0056266785,
 0.027389526,
 0.028289795,
 -0.022781372,
 0.021118164,
 -0.022994995,
 -0.049987793,
 0.0027751923,
 0.04925537,
 0.010757446,
 0.02684021,
 -0.020858765,
 0.023529053,
 -0.06488037,
 0.014717102,
 0.013954163,
 0.028549194,
 0.026428223,
 0.023391724,
 0.00060892105,
 0.002544403,
 0.023666382,
 -0.002866745,
 0.027145386,
 -0.00818634,
 0.002204895,
 0.0050354004,
 -0.0060272217,
 -0.0025501251,
 0.03152466,
 0.016815186,
 -0.034301758,
 0.04663086,
 -0.014587402,
 0.06756592,
 0.043914795,
 0.03741455,
 0.018051147,
 0.01948

In [71]:
cat = create_embeddings("cat")

cat

[0.00630188,
 0.010032654,
 -0.0020694733,
 -0.015914917,
 -0.02796936,
 0.007827759,
 -0.005268097,
 -0.036468506,
 -0.005508423,
 0.0060577393,
 0.023132324,
 0.013717651,
 0.0005393028,
 0.010406494,
 0.005924225,
 0.009422302,
 0.03390503,
 -0.021362305,
 0.010467529,
 -0.011383057,
 -0.0012874603,
 0.018737793,
 0.042938232,
 0.0024642944,
 -0.036132812,
 0.043304443,
 0.016082764,
 -0.036895752,
 0.018218994,
 -0.029144287,
 -0.016052246,
 -0.0056266785,
 0.027389526,
 0.028289795,
 -0.022781372,
 0.021118164,
 -0.022994995,
 -0.049987793,
 0.0027751923,
 0.04925537,
 0.010757446,
 0.02684021,
 -0.020858765,
 0.023529053,
 -0.06488037,
 0.014717102,
 0.013954163,
 0.028549194,
 0.026428223,
 0.023391724,
 0.00060892105,
 0.002544403,
 0.023666382,
 -0.002866745,
 0.027145386,
 -0.00818634,
 0.002204895,
 0.0050354004,
 -0.0060272217,
 -0.0025501251,
 0.03152466,
 0.016815186,
 -0.034301758,
 0.04663086,
 -0.014587402,
 0.06756592,
 0.043914795,
 0.03741455,
 0.018051147,
 0.01948

In [72]:
import pickle
from pathlib import Path

def save_embeddings(df, folder="embeddings", filename="flattened_df.pkl"):
    """
    Зберігає DataFrame з ембедінгами в указану папку.
    
    Args:
        df: DataFrame з ембедінгами
        folder: Назва папки для збереження
        filename: Ім'я файлу для збереження
    """
    # Створення директорії, якщо вона не існує
    Path(folder).mkdir(parents=True, exist_ok=True)
    
    # Шлях до файлу
    file_path = os.path.join(folder, filename)
    
    # Збереження DataFrame
    with open(file_path, 'wb') as f:
        pickle.dump(df, f)
    
    print(f"DataFrame успішно збережено в {file_path}")

def load_embeddings(folder="embeddings", filename="flattened_df.pkl"):
    """
    Завантажує DataFrame з ембедінгами з указаної папки.
    
    Args:
        folder: Назва папки для завантаження
        filename: Ім'я файлу для завантаження
        
    Returns:
        DataFrame з ембедінгами або None, якщо файл не існує
    """
    # Шлях до файлу
    file_path = os.path.join(folder, filename)
    
    # Перевірка існування файлу
    if os.path.exists(file_path):
        # Завантаження DataFrame
        with open(file_path, 'rb') as f:
            df = pickle.load(f)
        
        print(f"DataFrame успішно завантажено з {file_path}")
        return df
    else:
        print(f"Файл {file_path} не знайдено")
        return None

def get_or_create_embeddings(df, chunk_column, embedding_function, folder="embeddings", filename="flattened_df.pkl"):
    """
    Завантажує DataFrame з ембедінгами або створює новий.
    
    Args:
        df: Вихідний DataFrame з текстами
        chunk_column: Назва стовпця з текстовими фрагментами
        embedding_function: Функція для створення ембедінгів
        folder: Назва папки для збереження/завантаження
        filename: Ім'я файлу для збереження/завантаження
        
    Returns:
        DataFrame з ембедінгами
    """
    # Спроба завантажити DataFrame
    loaded_df = load_embeddings(folder, filename)
    
    if loaded_df is not None:
        return loaded_df
    
    # Якщо завантаження не вдалося, створюємо ембедінги
    print("Створення нових ембедінгів...")
    
    # Створення ембедінгів
    embeddings = []
    for chunk in df[chunk_column]:
        embeddings.append(embedding_function(chunk))
    
    # Збереження ембедінгів в DataFrame
    df['embeddings'] = embeddings
    
    # Збереження DataFrame
    save_embeddings(df, folder, filename)
    
    return df


# Використовуємо функцію, яка розраховує ембедінги, 
# якщо вони не були раніше створені і збережені в папці в "15-rag-and-vector-databases/embeddings"
flattened_df = get_or_create_embeddings(
    splitted_df.explode('chunks'), 
    'chunks', 
    create_embeddings
)

flattened_df.head()

Файл embeddings/flattened_df.pkl не знайдено
Створення нових ембедінгів...
DataFrame успішно збережено в embeddings/flattened_df.pkl


,path,text,chunks,embeddings
0,data/frameworks.md,# Neural Network Frameworks\n\nAs we have lear...,# Neural Network Frameworks As we have learned...,"[0.005393982, 0.018859863, 0.0063819885, 0.009..."
0,data/frameworks.md,# Neural Network Frameworks\n\nAs we have lear...,descent optimization While the `numpy` library...,"[0.01537323, 0.0284729, -0.02949524, 0.0531921..."
0,data/frameworks.md,# Neural Network Frameworks\n\nAs we have lear...,should give us the opportunity to compute grad...,"[0.016677856, -0.004032135, -0.01838684, 0.051..."
0,data/frameworks.md,# Neural Network Frameworks\n\nAs we have lear...,those computations on GPUs is very important. ...,"[-0.0016927719, -0.01461792, 0.013427734, 0.03..."
0,data/frameworks.md,# Neural Network Frameworks\n\nAs we have lear...,"API, there is also higher-level API, called Ke...","[-0.018341064, -0.008888245, -0.017364502, 0.0..."


# Пошук з використанням FAISS

Векторний пошук та схожість між нашим запитом і базою даних з використанням FAISS

### Створення індексу FAISS та підготовка до пошуку

In [73]:
# Отримуємо ембедінги як масив numpy
embeddings_list = flattened_df['embeddings'].to_list()
embeddings_array = np.array(embeddings_list).astype('float32')

# Визначаємо розмірність векторів
vector_dimension = len(embeddings_list[0])

# Створюємо індекс FAISS
index = faiss.IndexFlatL2(vector_dimension)  # L2 - це евклідова відстань

# Додаємо наші вектори до індексу
index.add(embeddings_array)

# Перевіряємо кількість векторів в індексі
print(f"Загальна кількість векторів в індексі: {index.ntotal}, розмірність векторів: {vector_dimension}")

Загальна кількість векторів в індексі: 57, розмірність векторів: 1024


In [74]:
# Ваше текстове запитання
question = "Що таке персептрон?"

# Перетворіть запитання у вектор запиту
query_vector = create_embeddings(question)

query_vector_array = np.array([query_vector]).astype('float32')

# Знайдіть найбільш схожі документи (k=5 - скільки найближчих сусідів шукаємо)
k = 5
distances, indices = index.search(query_vector_array, k)

# Виведіть найбільш схожі документи
for i in range(min(3, len(indices[0]))):
    idx = indices[0][i]
    print(f"Фрагмент {i+1}:")
    print(flattened_df['chunks'].iloc[idx])
    print(f"Шлях: {flattened_df['path'].iloc[idx]}")
    print(f"Відстань: {distances[0][i]}")
    print("-" * 50)

Фрагмент 1:
user to adjust the resistance of a circuit. > The New York Times wrote about perceptron at that time: *the embryo of an electronic computer that [the Navy] expects will be able to walk, talk, see, write, reproduce itself and be conscious of its existence.* ## Perceptron Model Suppose we have N features
Шлях: data/perceptron.md
Відстань: 1.0505201816558838
--------------------------------------------------
Фрагмент 2:
in our model, in which case the input vector would be a vector of size N. A perceptron is a **binary classification** model, i.e. it can distinguish between two classes of input data. We will assume that for each input vector x the output of our perceptron would be either +1 or -1, depending on the class.
Шлях: data/perceptron.md
Відстань: 1.1367818117141724
--------------------------------------------------
Фрагмент 3:
# Introduction to Neural Networks. Multi-Layered Perceptron In the previous section, you learned about the simplest neural network model - one-

## Поєднання всього для відповіді на запитання

In [75]:
from azure.ai.inference import ChatCompletionsClient

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

# Виберіть модель загального призначення для тексту
deployment = "gpt-4o-mini"

# Реалізація чатботів (при наявності і відсутності RAG)

In [13]:
def chatbot_with_rag(user_input):
    # Перетворіть запитання у вектор запиту
    query_vector = create_embeddings(user_input)
    query_vector_array = np.array([query_vector]).astype('float32')
    
    # Знайдіть найбільш схожі документи з FAISS
    k = 5  # кількість найближчих сусідів для пошуку
    distances, indices = index.search(query_vector_array, k)

    # додайте документи до запиту, щоб забезпечити контекст
    history = []
    for idx in indices[0]:
        history.append(flattened_df['chunks'].iloc[idx])

    # створюємо об'єкт повідомлення з контекстом
    context = "\n\n".join(history)  # всі знайдені фрагменти

    # Формуємо запит, що просить коротку, але завершену відповідь
    messages = [
        {"role": "system", "content": "You are an AI assistant that helps with AI questions. "},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {user_input}\n\n Provide a brief but complete answer based on the context. Answer in Ukrainian."}
    ]


    response = client.complete(
        temperature=0,
        model=deployment,
        messages=messages,
        max_tokens=300,
    )

    return response.choices[0].message.content


def chatbot_without_rag(user_input):
    """
    Чат-бот без використання RAG (прямий запит до моделі).
    """
    messages=[
        {"role": "system", "content": "You are an AI assistant that helps with AI questions. Provide brief but complete answers. Answer in Ukrainian."},
        {"role": "user", "content": f"Question: {user_input}"}
    ]

    response = client.complete(
        temperature=0,
        model=deployment,
        messages=messages,
        max_tokens=300,
    )

    return response.choices[0].message.content

# Порівняння відповідей RAG-системи

In [67]:
from IPython.display import display, Markdown, HTML

def compare_responses(user_input, save_to_file=False, filename="rag_comparison.md"):
    """
    Порівнює відповіді чат-боту з RAG та без RAG.
    
    Args:
        user_input: Запитання користувача
        save_to_file: Зберегти результат у файл Markdown
        filename: Назва файлу для збереження
    """
    # Отримання знайдених чанків
    query_vector = create_embeddings(user_input)

    query_vector_array = np.array([query_vector]).astype('float32')
    k = 5  # кількість найближчих сусідів для пошуку
    distances, indices = index.search(query_vector_array, k)
    
    # Отримання відповідей
    rag_response = chatbot_with_rag(user_input)
    no_rag_response = chatbot_without_rag(user_input)
    
    # Формування markdown-тексту
    markdown_text = f"""
# Порівняння відповідей

## 📝 Запит: {user_input}

## 🔍 Відповіді моделей

### Без використання RAG

{no_rag_response}

### З використанням RAG

{rag_response}

## 📚 Знайдені фрагменти тексту

"""
    
    # Додавання чанків
    for i, idx in enumerate(indices[0]):
        chunk_content = flattened_df['chunks'].iloc[idx]
        path = flattened_df['path'].iloc[idx]
        dist = float(distances[0][i])
        
        markdown_text += f"""
### Фрагмент {i+1} (відстань: {dist:.4f})

**Шлях**: {path}

{chunk_content}

"""
    
    # Виведення Markdown
    display(Markdown(markdown_text))
    
    # Для коректного відображення формул
    mathjax_script = """
    <script type="text/javascript">
        MathJax = {
            tex: {
                inlineMath: [['$', '$']]
            }
        };
    </script>
    <script type="text/javascript" id="MathJax-script" async
        src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-chtml.js">
    </script>
    """
    display(HTML(mathjax_script))
    
    # Збереження в файл, якщо потрібно
    if save_to_file:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(markdown_text)
        print(f"Результати збережено у файл: {filename}")


In [68]:
# Приклад використання:
compare_responses("Що таке перцептрон?")


# Порівняння відповідей

## 📝 Запит: Що таке перцептрон?

## 🔍 Відповіді моделей

### Без використання RAG

Перцептрон — це базова модель штучного нейрону, яка використовується в машинному навчанні для класифікації даних. Він складається з вхідних сигналів, ваг, активаційної функції та виходу. Перцептрон навчається шляхом корекції ваг на основі помилок, що виникають під час прогнозування. Це один з найпростіших видів нейронних мереж, який може вирішувати лінійно роздільні задачі.

### З використанням RAG

Перцептрон — це простий модельний алгоритм для бінарної класифікації, який використовує вектор ваг для класифікації вхідних даних. Він працює на основі формули y(x) = f(w<sup>T</sup>x), де f є кроковою активаційною функцією. Перцептрон навчається, коригуючи вектор ваг w, щоб зменшити помилку класифікації, яка визначається як E(w) = -&sum;w<sup>T</sup>x<sub>i</sub>t<sub>i</sub> для тих даних, які були неправильно класифіковані. Ця модель може бути використана для розв'язання як простих, так і реальних задач класифікації.

## 📚 Знайдені фрагменти тексту


### Фрагмент 1 (відстань: 0.3923)

**Шлях**: data/perceptron.md

by **perceptron criterion** in the following manner: E(w) = -&sum;w<sup>T</sup>x<sub>i</sub>t<sub>i</sub> where: * the sum is taken on those training data points i that result in the wrong classification * x<sub>i</sub> is the input data, and t<sub>i</sub> is either -1 or +1 for negative and positive


### Фрагмент 2 (відстань: 0.3938)

**Шлях**: data/perceptron.md

user to adjust the resistance of a circuit. > The New York Times wrote about perceptron at that time: *the embryo of an electronic computer that [the Navy] expects will be able to walk, talk, see, write, reproduce itself and be conscious of its existence.* ## Perceptron Model Suppose we have N features


### Фрагмент 3 (відстань: 0.3984)

**Шлях**: data/perceptron.md

and to continue learning - go to Perceptron notebook. Here's an interesting article about perceptrons as well. ## Assignment In this lesson, we have implemented a perceptron for binary classification task, and we have used it to classify between two handwritten digits. In this lab, you are asked to solve


### Фрагмент 4 (відстань: 0.4032)

**Шлях**: data/perceptron.md

classification model, and how to train it by using a weights vector. ## 🚀 Challenge If you'd like to try to build your own perceptron, try this lab on Microsoft Learn which uses the Azure ML designer ## Review & Self Study To see how we can use perceptron to solve a toy problem as well as real-life problems,


### Фрагмент 5 (відстань: 0.4034)

**Шлях**: data/perceptron.md

The output will be computed using the formula: y(x) = f(w<sup>T</sup>x) where f is a step activation function ## Training the Perceptron To train a perceptron we need to find a weights vector w that classifies most of the values correctly, i.e. results in the smallest **error**. This error is defined

